In [2]:
import pandas as pd
import os
from nba_api.stats.endpoints import playerindex, playbyplayv3
import utils.data_utils as du
import importlib


In [7]:
HOME_DIR = '/Users/yoav/Documents/Yoav/CS/20936 - DS Project/Shotpred'
DATA_DIR = os.path.join(HOME_DIR, 'data')
EXT_DATA_DIR = os.path.join(DATA_DIR, 'External')
INT_DATA_DIR = os.path.join(DATA_DIR, 'Internal')

In [8]:
KAGGLE_SHOT_LOGS_201415_FILENAME = 'kaggle_shot_logs_2014_15.csv'
KAGGLE_SHOT_LOGS_201415_FULLPATH = os.path.join(EXT_DATA_DIR, KAGGLE_SHOT_LOGS_201415_FILENAME)

GENERAL_SHOT_LOGS_FILENAME = 'nba_2015_shots.csv'
GENERAL_SHOT_LOGS_FULLPATH = os.path.join(EXT_DATA_DIR, GENERAL_SHOT_LOGS_FILENAME)

In [36]:
KAGGLE_COLS_FOR_MERGE = ['GAME_ID', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME', 'CLOSEST_DEFENDER', 'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST', 'player_id', 'ABS_TIME']
CAREER_STATS_COL_FOR_MERGE = [['PLAYER_ID', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT']] #already filtered for relevant season

In [9]:
shot_logs_1415 = pd.read_csv(KAGGLE_SHOT_LOGS_201415_FULLPATH)

In [10]:
active_players_201415_ids = shot_logs_1415['player_id'].unique()

In [11]:
shot_logs_1415['ABS_TIME'] = shot_logs_1415.apply(lambda row: du.clock_to_absolute_seconds(row['GAME_CLOCK'], row['PERIOD']), axis=1)

In [20]:
#active_players_career_stats = du.gather_mul_career_stats(active_players_201415_ids) # get the career stats for all active players in 2014-15 season

In [36]:
#active_players_cs_in_1415 = active_players_career_stats[active_players_career_stats['SEASON_ID'] == '2014-15'].reset_index(drop=True)
#active_players_cs_in_1415.to_csv(INT_DATA_DIR + 'active_players_career_stats_2014_15.csv', index=False)

In [13]:
active_players_cs_in_1415 = pd.read_csv(os.path.join(INT_DATA_DIR, 'active_players_career_stats_2014_15.csv'))

In [14]:
gen_sl = pd.read_csv(GENERAL_SHOT_LOGS_FULLPATH)

In [54]:
gen_sl.head()

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2015,2014-15,1610612743,Denver Nuggets,203115,Will Barton,G,SG,04-15-2015,21401229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.4,6.75,1,4,0,18
1,2015,2014-15,1610612744,Golden State Warriors,203949,James Michael McAdoo,F,PF,04-15-2015,21401229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.35,0,4,0,22
2,2015,2014-15,1610612739,Cleveland Cavaliers,1890,Shawn Marion,F,SF,04-15-2015,21401225,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.35,0,5,0,3
3,2015,2014-15,1610612744,Golden State Warriors,203949,James Michael McAdoo,F,PF,04-15-2015,21401229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.35,0,4,0,37
4,2015,2014-15,1610612743,Denver Nuggets,203475,Erick Green,G,PG,04-15-2015,21401229,...,Mid-Range,Left Side,L,16-24 ft.,16.9,14.85,19,4,0,48


In [ ]:
gen_sl['ABS_TIME'] = gen_sl.apply(lambda row: du.clock_to_absolute_seconds(f"{str(row["MINS_LEFT"]).zfill(2)}:{str(row["SECS_LEFT"]).zfill(2)}", row['QUARTER']), axis=1)

In [37]:
kaggle_for_join = shot_logs_1415[KAGGLE_COLS_FOR_MERGE]

In [42]:
kaggle_for_join.GAME_ID.unique()

array([21400899, 21400890, 21400882, 21400859, 21400845, 21400827,
       21400816, 21400783, 21400768, 21400761, 21400742, 21400719,
       21400713, 21400689, 21400652, 21400643, 21400625, 21400611,
       21400600, 21400574, 21400549, 21400535, 21400522, 21400510,
       21400495, 21400484, 21400478, 21400459, 21400444, 21400438,
       21400423, 21400409, 21400395, 21400384, 21400369, 21400355,
       21400341, 21400334, 21400316, 21400276, 21400263, 21400237,
       21400224, 21400210, 21400198, 21400195, 21400172, 21400160,
       21400146, 21400140, 21400130, 21400107, 21400097, 21400071,
       21400057, 21400054, 21400041, 21400028, 21400004, 21400902,
       21400888, 21400873, 21400862, 21400848, 21400808, 21400898,
       21400894, 21400876, 21400866, 21400851, 21400836, 21400812,
       21400797, 21400780, 21400764, 21400756, 21400740, 21400729,
       21400703, 21400690, 21400673, 21400655, 21400638, 21400626,
       21400610, 21400597, 21400571, 21400555, 21400542, 21400

In [43]:
gen_sl.GAME_ID.unique()

array([21401229, 21401225, 21401230, ..., 21400003, 21400001, 21400002])

In [44]:
gen_sl_game_ids = set(gen_sl['GAME_ID'].unique())
kaggle_for_join_game_ids = set(kaggle_for_join['GAME_ID'].unique())

common_game_ids = gen_sl_game_ids.intersection(kaggle_for_join_game_ids)

print(f"Number of unique GAME_IDs in gen_sl: {len(gen_sl_game_ids)}")
print(f"Number of unique GAME_IDs in kaggle_for_join: {len(kaggle_for_join_game_ids)}")
print(f"Number of common GAME_IDs in both: {len(common_game_ids)}")

Number of unique GAME_IDs in gen_sl: 1230
Number of unique GAME_IDs in kaggle_for_join: 904
Number of common GAME_IDs in both: 904


In [38]:
base_and_kag = pd.merge(gen_sl, kaggle_for_join, on=['GAME_ID', 'ABS_TIME'], suffixes=('_gen', '_kag'))
base_and_kag.head()

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,MINS_LEFT,SECS_LEFT,ABS_TIME,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_id
0,2015,2014-15,1610612760,Oklahoma City Thunder,201566,Russell Westbrook,G,PG,03-04-2015,21400904,...,1,55,3485.0,20.2,5,4.9,"Noel, Nerlens",203457,0.0,201566
1,2015,2014-15,1610612757,Portland Trail Blazers,200746,LaMarcus Aldridge,F,PF,03-04-2015,21400908,...,0,59,3541.0,12.9,0,0.4,"Jordan, DeAndre",201599,1.9,200746
2,2015,2014-15,1610612747,Los Angeles Lakers,201941,Jordan Hill,C,C,03-04-2015,21400905,...,0,0,2880.0,NaN,0,0.9,"Walker, Henry",201611,16.4,201941
3,2015,2014-15,1610612738,Boston Celtics,203092,Tyler Zeller,C,C,03-04-2015,21400898,...,0,0,2880.0,NaN,0,1.4,"Hood, Rodney",203918,1.7,203092
4,2015,2014-15,1610612751,Brooklyn Nets,202721,Darius Morris,G,PG,03-04-2015,21400899,...,2,5,2755.0,24.0,0,1.5,"Vonleh, Noah",203943,2.5,202366


In [39]:
gen_sl.shape, kaggle_for_join.shape, base_and_kag.shape

((205550, 27), (128069, 9), (5571, 34))

In [35]:
gen_sl.groupby('ABS_TIME').size().sort_values(ascending=False)

ABS_TIME
720.0     756
1440.0    754
2160.0    751
2159.0    326
719.0     325
         ... 
4173.0      1
4176.0      1
4178.0      1
4179.0      1
5038.0      1
Length: 3386, dtype: int64

In [45]:
ap_1415_index = playerindex.PlayerIndex(season='2014-15').get_data_frames()[0]

In [ ]:
ap_1415_index_for_join = ap_1415_index[['PERSON_ID', 'HEIGHT']]

In [56]:
test = playbyplayv3.PlayByPlayV3(game_id='0021401229').get_data_frames()[0]

In [ ]:
test.actionType.unique()

In [69]:
kk = test[test.actionType == 'Substitution'].reset_index(drop=True)

In [123]:
kk.head()

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId
0,0021401229,73,PT04M45.00S,1,1610612744,GSW,202691,Thompson,K. Thompson,0,...,0,0,0,h,SUB: Iguodala FOR Thompson,Substitution,,0,0,73
1,0021401229,74,PT04M45.00S,1,1610612744,GSW,203084,Barnes,H. Barnes,0,...,0,0,0,h,SUB: Barbosa FOR Barnes,Substitution,,0,0,74
2,0021401229,75,PT04M45.00S,1,1610612743,DEN,203994,Nurkic,J. Nurkic,0,...,0,0,0,v,SUB: Hickson FOR Nurkic,Substitution,,0,0,75
3,0021401229,103,PT02M12.00S,1,1610612744,GSW,101106,Bogut,A. Bogut,0,...,0,0,0,h,SUB: Ezeli FOR Bogut,Substitution,,0,0,100
4,0021401229,104,PT02M12.00S,1,1610612743,DEN,203914,Harris,G. Harris,0,...,0,0,0,v,SUB: Barton FOR Harris,Substitution,,0,0,101


In [134]:
test['ABS_TIME'] = test.apply(lambda row: du.clock_to_absolute_seconds(row['clock'], row['period']), axis=1)

In [149]:
shot_logs_1415.head() # kaggle shot logs

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,ABS_TIME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148,651.0
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148,706.0
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148,720.0
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148,733.0
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148,806.0


In [152]:
shot_logs_1415.columns

Index(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN', 'SHOT_NUMBER',
       'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',
       'SHOT_DIST', 'PTS_TYPE', 'SHOT_RESULT', 'CLOSEST_DEFENDER',
       'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST', 'FGM', 'PTS',
       'player_name', 'player_id', 'ABS_TIME'],
      dtype='object')

In [159]:
shot_logs_1415.head(3)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,ABS_TIME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148,651.0
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148,706.0
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148,720.0


In [160]:
active_players_cs_in_1415.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [151]:
gen_sl.columns

Index(['SEASON_1', 'SEASON_2', 'TEAM_ID', 'TEAM_NAME', 'PLAYER_ID',
       'PLAYER_NAME', 'POSITION_GROUP', 'POSITION', 'GAME_DATE', 'GAME_ID',
       'HOME_TEAM', 'AWAY_TEAM', 'EVENT_TYPE', 'SHOT_MADE', 'ACTION_TYPE',
       'SHOT_TYPE', 'BASIC_ZONE', 'ZONE_NAME', 'ZONE_ABB', 'ZONE_RANGE',
       'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT'],
      dtype='object')

In [153]:
test.columns

Index(['gameId', 'actionNumber', 'clock', 'period', 'teamId', 'teamTricode',
       'personId', 'playerName', 'playerNameI', 'xLegacy', 'yLegacy',
       'shotDistance', 'shotResult', 'isFieldGoal', 'scoreHome', 'scoreAway',
       'pointsTotal', 'location', 'description', 'actionType', 'subType',
       'videoAvailable', 'shotValue', 'actionId', 'ABS_TIME'],
      dtype='object')